In [211]:
import ast
def print_ast(src):
    print(ast.dump(src, indent=4))
def print_code(src):
    print(ast.unparse(ast.fix_missing_locations(src)))

In [212]:
def simple_function(x):
    y = 3 * x
    print(y)

In [213]:
import inspect

In [214]:
tree = ast.parse(inspect.getsource(simple_function))

In [215]:
print_ast(tree)

Module(
    body=[
        FunctionDef(
            name='simple_function',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='y', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=3),
                        op=Mult(),
                        right=Name(id='x', ctx=Load()))),
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[]))],
            decorator_list=[])],
    type_ignores=[])


In [216]:
print_ast(tree.body[0])

FunctionDef(
    name='simple_function',
    args=arguments(
        posonlyargs=[],
        args=[
            arg(arg='x')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
    body=[
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=BinOp(
                left=Constant(value=3),
                op=Mult(),
                right=Name(id='x', ctx=Load()))),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='y', ctx=Load())],
                keywords=[]))],
    decorator_list=[])


In [217]:
import random

We create a PythonMutator class that links to other Mutator classes, the PythonMutator class tying all of them together with helper functions to call those class' methods. We can later use it to add probabilities etc. to each mutation.

In [218]:
class PythonMutator:
    def visit_Module(self, src):
        return self.generic_visit(src)
    
    def visit_FunctionDef(self, src):
        return self.generic_visit(src)
    
    def visit_BinOp(self, src):
        return self.generic_visit(src)
    
    def visit_Assign(self, src):
        return self.generic_visit(src)
    
    def visit_Call(self, src):
        return self.generic_visit(src)
    
    def visit_Name(self, src):
        return self.generic_visit(src)
    
    def visit_Constant(self, src):
        return self.generic_visit(src)

    def expand_constants(self, src):
        return ExprMutator().modify_value(src)
    
    def swap_numbers(self, src):
        return ExprMutator().commute_value(src)

Now we need to define modify_value that can replace a given constant with an equivalent arithmetic expression, and swap_numbers that will swap the children of a + or * node.

In [219]:
op_map = [("+", ast.Add()), ("*", ast.Mult()), ("/", ast.Div()), ("-", ast.Sub())]

In [220]:
class ExprMutator(ast.NodeTransformer):
    EXPAND = 1
    COMMUTE = 2

    def __init__(self):
        self.transform = False
        self.depth = 0
        self.mode = self.EXPAND

    def modify_value(self, n, depth=3):
        self.mode = self.EXPAND
        self.depth = depth
        return self._modify_value(n)
    
    def _modify_value(self, n):
        if self.depth == 0: return n
        self.transform = True
        return self._modify_value(self.visit(n))

We need the mode so we can swap between traversing a path and swapping children. Depth allows us to control how many numbers we want to go and replace with expressions.

In [221]:
class ExprMutator(ExprMutator):    
    def commute_value(self, n):
        self.mode = self.COMMUTE
        return self.visit(n)

Now come the real functions. The visits to Constant or BinOp nodes are what will truly handle the functionality.

In [222]:
class ExprMutator(ExprMutator):
    def visit_Constant(self, src):
        if isinstance(src.value, int) and self.transform and self.mode == self.EXPAND:
            while True:
                try:
                    op = random.randint(0, 3)
                    other = random.randint(-10000, 10000)
                    assert eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")" + op_map[op][0] + str(other)) == src.value
                    break
                except ZeroDivisionError: continue
                except AssertionError: continue
            self.depth -= 1
            self.transform = False
            return ast.BinOp(left = ast.Constant(value=eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")")), op = op_map[op][1], right = ast.Constant(value=other))
            
        return src

    def visit_BinOp(self, src):
        if self.mode == self.EXPAND:
            if random.randint(1, 2) == 1:
                src.left = self.visit(src.left)
            else:
                src.right = self.visit(src.right)
            return src
        
        if self.mode == self.COMMUTE:
            if isinstance(src.op, ast.Add) or isinstance(src.op, ast.Mult):
                src.left, src.right = src.right, src.left
                
            return self.generic_visit(src)

In [223]:
print_ast(ExprMutator().modify_value(ast.Constant(value=1), depth=2))

BinOp(
    left=BinOp(
        left=Constant(value=5400),
        op=Add(),
        right=Constant(value=-9000)),
    op=Div(),
    right=Constant(value=-3600))


In [224]:
print_code(PythonMutator().expand_constants(ast.parse("x+1")))

x + (2.1741573033707864 * (-1.050354051927616 * 2542) - -5806)


In [225]:
from copy import deepcopy

In [226]:
print_code(PythonMutator().swap_numbers(ast.parse("x + 0.00042426813746287653 * (-5.193317422434368 * 1257 + 8885)")))

(8885 + 1257 * -5.193317422434368) * 0.00042426813746287653 + x


In [227]:
tree_two = deepcopy(tree)
for i in range(5):
    if random.randint(1, 5) == 1: PythonMutator().swap_numbers(tree_two)
    else: PythonMutator().expand_constants(tree_two)
new_code = ast.unparse(tree_two)
print(new_code)

def simple_function(x):
    y = x * ((-19873 - -8326 - 1963 - -0.6695849914724276 * 8795) * 1.093688492323842 * -0.00035992801439712057)
    print(y)


In [228]:
simple_function(456)

1368


In [229]:
exec(new_code)

In [230]:
simple_function(456)

1368.0


Clearly the output remains the same inspite of our changes. Next, we look into transforming range-based for loops into while loops.

In [231]:
for_tree = ast.parse('''for i in range(10, 1, -2):
                        print(i)''')
print_ast(for_tree)

Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
                args=[
                    Constant(value=10),
                    Constant(value=1),
                    UnaryOp(
                        op=USub(),
                        operand=Constant(value=2))],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])


In [232]:
while_tree = ast.parse('''
i = 10
while i > 1:
    print(i)
    i += -2''')
print_ast(while_tree)

Module(
    body=[
        Assign(
            targets=[
                Name(id='i', ctx=Store())],
            value=Constant(value=10)),
        While(
            test=Compare(
                left=Name(id='i', ctx=Load()),
                ops=[
                    Gt()],
                comparators=[
                    Constant(value=1)]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[])),
                AugAssign(
                    target=Name(id='i', ctx=Store()),
                    op=Add(),
                    value=UnaryOp(
                        op=USub(),
                        operand=Constant(value=2)))],
            orelse=[])],
    type_ignores=[])


In [233]:
src = for_tree.body[0]
def analyze_for(node):
    args = node.iter.args
    if len(args) == 1:
        return [ast.Constant(value=0), ast.Lt(), args[0], ast.Constant(value=1)]
    elif len(args) == 2:
        return [args[0], ast.Lt(), args[1], ast.Constant(value=1)]
    else:
        step = eval(ast.unparse(args[2]))
        if step < 0:
            return [args[0], ast.Gt(), args[1], args[2]]
        else:
            return [args[0], ast.Lt(), args[1], args[2]]
        
while_args = analyze_for(src)
print_ast(
    ast.Assign(targets=[src.target], value=while_args[0])
    ) 
print_ast(
    ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
              body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)
)

Assign(
    targets=[
        Name(id='i', ctx=Store())],
    value=Constant(value=10))
While(
    test=Compare(
        left=Name(id='i', ctx=Load()),
        ops=[
            Gt()],
        comparators=[
            Constant(value=1)]),
    body=[
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='i', ctx=Load())],
                keywords=[])),
        AugAssign(
            target=Name(id='i', ctx=Store()),
            op=Add(),
            value=UnaryOp(
                op=USub(),
                operand=Constant(value=2)))],
    orelse=[])


In [234]:
class PythonMutator(PythonMutator):
    def transform_for(self, src):
        return ForMutator().visit(src)

In [235]:
class ForMutator(ast.NodeTransformer):
    def visit_For(self, src):  
        if isinstance(src.iter, ast.Name): return src
        while_args = analyze_for(src)

        return [ast.Assign(targets=[src.target], value=while_args[0]), \
                ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
                          body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)]

In [236]:
for_tree_two = deepcopy(for_tree)
print(ast.unparse(for_tree))
print("====")
print_code(PythonMutator().transform_for(for_tree_two))

for i in range(10, 1, -2):
    print(i)
====
i = 10
while i > 1:
    print(i)
    i += -2


That takes care of for-loops based on ranges. <b>What about iterators?</b>

In [237]:
print_ast(ast.parse(
'''
L = [1, 4, "hello"]
for i in [len(str(x)) for x in L]:
    print(i)
'''
))

Module(
    body=[
        Assign(
            targets=[
                Name(id='L', ctx=Store())],
            value=List(
                elts=[
                    Constant(value=1),
                    Constant(value=4),
                    Constant(value='hello')],
                ctx=Load())),
        For(
            target=Name(id='i', ctx=Store()),
            iter=ListComp(
                elt=Call(
                    func=Name(id='len', ctx=Load()),
                    args=[
                        Call(
                            func=Name(id='str', ctx=Load()),
                            args=[
                                Name(id='x', ctx=Load())],
                            keywords=[])],
                    keywords=[]),
                generators=[
                    comprehension(
                        target=Name(id='x', ctx=Store()),
                        iter=Name(id='L', ctx=Load()),
                        ifs=[],
                        is_async=0)

In [238]:
print_ast(ast.parse('x,y = 3 + 5, 3 + 5'))

Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5))],
                ctx=Load()))],
    type_ignores=[])


In [239]:
print_ast(ast.parse('''
tmp1, tmp2 = 3 + 5, 3 + 5
x, y = tmp1, tmp2'''))

Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='tmp1', ctx=Store()),
                        Name(id='tmp2', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5))],
                ctx=Load())),
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    Name(id='tmp1', ctx=Load()),
                    Name(id='

In [240]:
class AssignMutator(ast.NodeTransformer):
    def __init__(self):
        self.in_assign = False

    def visit_Name(self, src):
        if self.in_assign: return ast.Name(id = '_' + str(random.randint(1087345, 196871238674)), ctx = src.ctx)
        return src
     
    def visit_Assign(self, src):
        self.in_assign = True
        new_target = self.visit(deepcopy(src.targets[0]))
        self.in_assign = False
        return [ast.Assign(targets = [new_target], value=src.value), ast.Assign(targets=src.targets, value=NameHandler().get_name(deepcopy(new_target)))]

The naive method is to copy all targets and rename them on a line above. 

List and Dict subscripts are an issue for this. For example the following AST:

Assign(
    targets=[
        Subscript(
            value=Name(id='gates', ctx=Load()),
            slice=Subscript(
                value=Name(id='inps', ctx=Load()),
                slice=Constant(value=0),
                ctx=Load()),
            ctx=Store())],
    value=Call(
        func=Name(id='float', ctx=Load()),
        args=[
            Subscript(
                value=Name(id='inps', ctx=Load()),
                slice=Constant(value=1),
                ctx=Load())],
        keywords=[]
        )
)

representing 

gates[inps[0]] = float(inps[1])

gets transformed to

xxx[yyy[0]] = float(inps[1])
gates[inps[0]] = xxx[yyy[0]]

but this is problematic because xxx and yyy aren't declared as lists, which they need to be.

To get around this, we need to transform each target to a single variable node.

x, y = 5, 3

must be transformed to

tmp = 5, 3
x, y = tmp

instead.

In [241]:
class AssignMutator(ast.NodeTransformer):
    def visit_Assign(self, src):
        new_target = ast.Name(id = '_' + str(random.randint(1087345, 196871238674)), ctx = ast.Store())
        return [ast.Assign(targets = [new_target], value=src.value), ast.Assign(targets=src.targets, value=NameHandler().get_name(deepcopy(new_target)))]

In [242]:
class NameHandler(ast.NodeTransformer):
    def get_name(self, src, mode="LOAD"):
        self.mode = mode
        return self.visit(src)

    def visit_Name(self, src):
        if self.mode == "LOAD":
            return ast.Name(id = src.id, ctx = ast.Load())

In [243]:
print_ast(AssignMutator().visit(ast.parse('x,y = 3 + 5, 5 + 3')))

Module(
    body=[
        Assign(
            targets=[
                Name(id='_135867955336', ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=5),
                        op=Add(),
                        right=Constant(value=3))],
                ctx=Load())),
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Name(id='_135867955336', ctx=Load()))],
    type_ignores=[])


In [244]:
print_ast(ast.parse('x=y=5'))

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store()),
                Name(id='y', ctx=Store())],
            value=Constant(value=5))],
    type_ignores=[])


In [245]:
print_ast(AssignMutator().visit(ast.parse('x=y=5')))

Module(
    body=[
        Assign(
            targets=[
                Name(id='_16702392597', ctx=Store())],
            value=Constant(value=5)),
        Assign(
            targets=[
                Name(id='x', ctx=Store()),
                Name(id='y', ctx=Store())],
            value=Name(id='_16702392597', ctx=Load()))],
    type_ignores=[])


In [246]:
print_code(AssignMutator().visit(ast.parse('x,y = 3 + 5, 5 + 3')))

_24335968789 = (3 + 5, 5 + 3)
x, y = _24335968789


In [247]:
print_code(AssignMutator().visit(ast.parse('x=y=5')))

_164633165409 = 5
x = y = _164633165409


In [248]:
class PythonMutator(PythonMutator):
    def transform_assign(self, src):
        return AssignMutator().visit(src)

In [249]:
tree = ast.parse(r'''
with open("circuit.txt", "r") as F:
    circuit = F.readlines() # read circuit file into a list
with open("gate_delays.txt", "r") as F:
    delays = F.readlines() # read gate delays into a list

gates = {-1: 0} # prepare dictionary to allow simpler access of gate delays
nodes = {} # prepare dictionary to store node data
out_nodes = [] # prepare list to store names of output nodes
flag1 = flag2 = flag3 = False # prep for processing circuit later

# loop to assign delay value to each kind of gate
for i in delays:
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    gates[inps[0]] = float(inps[1]) # assign corresponding delay values with key as gate name

for i in circuit:
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    if inps[0] == "PRIMARY_INPUTS": # handling input signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate  
        flag1 = True # flag to say input signals have been read
        continue
    if inps[0] == "INTERNAL_SIGNALS": # handling internal signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate
        flag2 = True # flag to say internal signals have been read
        continue
    if inps[0] == "PRIMARY_OUTPUTS": # handling output signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate
        out_nodes.extend(inps[1:]) # list of output nodes
        flag3 = True # flag to say output signals have been read
        continue
    if flag1 and flag2 and flag3: break # break the loop if all 3 conditions are met before loop termination

for i in circuit: # processing the input and setting up input nodes and gates for each node
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    if ((inps[0]=="PRIMARY_INPUTS") or (inps[0]=="INTERNAL_SIGNALS") or (inps[0]=="PRIMARY_OUTPUTS")): 
        continue # ignore signal lines
    out = inps[-1]
    nodes[out][1].extend(inps[1:-1]) # set up input nodes for each node
    nodes[out][2] = inps[0] # set gate delay for relevant nodes

def calcVal_A(x): # recursive function to calculate the delay at each node
    # print(x, nodes) # debug line
    if nodes[x][1] == []: return nodes[x][0] # skip recursive step if node already processed
    s = 0
    for i in nodes[x][1]: # find max delay time of each input node
        nodes[i][0] = calcVal_A(i) # recursive call to function
        s = max(nodes[i][0], s) # node delay that controls delay time of output
    nodes[x][1] = [] # clear input nodes to indicate node delay is already calculated
    return s + gates[nodes[x][2]] # gate delay compensation

to_write = [] # initialize array of lines to be written to output

for i in out_nodes:
    nodes[i][0] = calcVal_A(i) # calculate delay for each output node using the recursive function
    if nodes[i][0] == round(nodes[i][0]): nodes[i][0] = round(nodes[i][0])
    to_write.append(i + " " + str(nodes[i][0]) + "\n") # write delay at each output node to array

with open("output_delays.txt", "w") as F:
    F.writelines(to_write) # write output array to file
''')

In [250]:
print_code(PythonMutator().expand_constants(PythonMutator().transform_assign(tree)))

with open('circuit.txt', 'r') as F:
    _132596022445 = F.readlines()
    circuit = _132596022445
with open('gate_delays.txt', 'r') as F:
    _155561413091 = F.readlines()
    delays = _155561413091
_63344760261 = {-(-1799 - (-2024901 + 5301) / 1122): 0}
gates = _63344760261
_130514500 = {}
nodes = _130514500
_137572938915 = []
out_nodes = _137572938915
_58587889275 = False
flag1 = flag2 = flag3 = _58587889275
for i in delays:
    _45757625402 = i.strip()
    x = _45757625402
    if x[:2] == '//':
        continue
    if len(x) == 0:
        continue
    _102067739930 = x.split()
    inps = _102067739930
    _67542697727 = float(inps[1])
    gates[inps[0]] = _67542697727
for i in circuit:
    _128379100702 = i.strip()
    x = _128379100702
    if x[:2] == '//':
        continue
    if len(x) == 0:
        continue
    _60055845878 = x.split()
    inps = _60055845878
    if inps[0] == 'PRIMARY_INPUTS':
        for j in inps[1:]:
            _61568351820 = [0, [], -1]
            nodes[j

In [251]:
import trace
import sys

In [252]:
def traceit(frame, event, arg):
    """Trace program execution. To be passed to sys.settrace()."""
    if event == 'line':
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        vars = dict(frame.f_locals)
        coverage.append([function_name, lineno, vars])
    return traceit

def tracer(f):
    global coverage
    coverage = []
    sys.settrace(traceit)  # Turn on
    f()
    sys.settrace(None)    # Turn off

In [253]:
def g():
    def simple_function(x):
        z = 2
        y = 3 * x
        return y
    
    a = simple_function(2)
    b = 0
    for _ in range(6):
        b += 2 * a

    print("The answer is", b)

In [254]:
tracer(g)

The answer is 72


In [255]:
for i in coverage:
    print(f"{i[0]} {i[1]} {i[2]}")

g 2 {}
g 7 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>}
simple_function 3 {'x': 2}
simple_function 4 {'x': 2, 'z': 2}
simple_function 5 {'x': 2, 'z': 2, 'y': 6}
g 8 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 0}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 0, '_': 0}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 12, '_': 0}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 12, '_': 1}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 24, '_': 1}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000001BD8717F240>, 'a': 6, 'b': 24, '_': 2}
g 9 {'simple_function': <function g.<locals>.simple_funct

In [256]:
g_tree = ast.parse(inspect.getsource(g)).body[0]
print_ast(g_tree)

FunctionDef(
    name='g',
    args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
    body=[
        FunctionDef(
            name='simple_function',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='z', ctx=Store())],
                    value=Constant(value=2)),
                Assign(
                    targets=[
                        Name(id='y', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=3),
                        op=Mult(),
                        right=Name(id='x', ctx=Load()))),
                Return(
                    value=Name(id='y', ctx=Load()))],
            decorator_list=[]),
   

In [257]:
print_ast(ast.parse('def f(x, y, *, z=3): print(x)'))

Module(
    body=[
        FunctionDef(
            name='f',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x'),
                    arg(arg='y')],
                kwonlyargs=[
                    arg(arg='z')],
                kw_defaults=[
                    Constant(value=3)],
                defaults=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='x', ctx=Load())],
                        keywords=[]))],
            decorator_list=[])],
    type_ignores=[])


In [258]:
for node in g_tree.body:
    print(node.lineno)

2
7
8
9
12


In [259]:
def get_trace(src):
    for node in src.body:
        data = src.name, node.lineno
        print(data)
        if isinstance(node, ast.FunctionDef):
            get_trace(node)

In [260]:
get_trace(g_tree)

('g', 2)
('simple_function', 3)
('simple_function', 4)
('simple_function', 5)
('g', 7)
('g', 8)
('g', 9)
('g', 12)


Now that we can get the line data for each node in the AST, we can get the data of the local variables at a particular AST node and use it for substitutions.

In [261]:
class VariableInjector(ast.NodeTransformer):
    def traceit(self, frame, event, arg):
        if event == 'line':
            function_name = frame.f_code.co_name
            lineno = frame.f_lineno
            vars = dict(frame.f_locals)
            self.coverage.append([function_name, lineno, vars])
        return self.traceit

    def tracer(self, f):
        self.coverage = []
        sys.settrace(self.traceit)  # Turn on
        f()
        sys.settrace(None)    # Turn off

    def profile_function(self, f):
        fn_tree = ast.parse(inspect.getsource(f)).body[0]
        self.tracer(f)
        self.visit(fn_tree)
        return fn_tree
    
                 

Currenly our class simply combines our existing methods, and then visits the AST. Now what we have to do is, while visiting the AST, we need to find the in-scope variables and their values at every line of execution. Then, we need to look for constants and check if they can be replaced by some variable or some simple arithmetic expression involving a variable.

In [262]:
class VariableInjector(VariableInjector):
    def visit_FunctionDef(self, src):
        self.args = [x.arg for x in src.args.args + src.args.kwonlyargs]
        for node in src.body:
            self.get_locals(src.name, node.lineno)
            self.visit(node)
                
    def visit_Constant(self, src):
        if len(self.local_vars) == 0: return src
        for var in self.local_vars.keys():
            if src.value == self.local_vars[var]:
                return ast.Name(id=var, ctx=ast.Load())
            elif isinstance(src.value, int) and isinstance(self.local_vars[var], int):
                try:
                    op = random.randint(0, 3)
                    other = self.local_vars[var]
                    assert eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")" + op_map[op][0] + str(other)) == src.value
                    return ast.BinOp(left = ast.Constant(value=eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")")), op = op_map[op][1], right = ast.Name(id=var, ctx=ast.Load())) 
                except ZeroDivisionError: continue
                except AssertionError: continue
            
        return src

We have written the functions that traverse the tree and make appropriate calls to functions to get our local variables. Since we are running this entire thing on a function, the outermost scope will always be handled, and then similiarly inner scopes will get handled. One thing we should note is, when using get_locals, we should avoid substituting constants with arguments to the function, because it won't be consistent across function calls.

In [263]:
class VariableInjector(VariableInjector):
    def get_locals(self, fn, ln):
        self.local_vars = {}
        for i in self.coverage:
            if i[0] == fn and i[1] == ln:
                self.local_vars = {k: v for k, v in i[2].items() if k not in self.args}
                return

In [269]:
print_code(ast.parse(inspect.getsource(g)))

def g():

    def simple_function(x):
        z = 2
        y = 3 * x
        return y
    a = simple_function(2)
    b = 0
    for _ in range(6):
        b += 2 * a
    print('The answer is', b)


In [270]:
print_code(VariableInjector().profile_function(g))

The answer is 72
def g():

    def simple_function(x):
        z = 2
        y = (1 + z) * x
        return y
    a = simple_function(2)
    b = -6 + a
    for _ in range(a):
        b += (-4 + a) * a
    print('The answer is', b)
